# Turbofan Tutorial
`pyturbo`  library is provided by twiinIT to assembly a simple turbofan system.

The library is made of components: 

- `compressor` : fluid out is computed from fluid in and power provided by shaft with constant efficiency. 
- `combustor` : combustion is made considering constant FHV.
- `turbine` : power is extracted from fluid in considering a given expansion ratio and constant efficiency. 
- `inlet` and `nozzle` are computing `drag` and `thrust` from fluid conditions, ambiant pressure and throat section. 
- `nacelle`: envelop over the engine.
- `ogv`, `intermediate_casing` and `trf` and structures with aero channels. 

They are numerical components:

- `fluid_spitter` is used to split the flow into primary and secondary flow
- `shaft_spitter` is used to split the shaft power into booster and fan compressor

A turbofan system is generated.

In [ ]:
from pyturbo.systems import Turbofan

tf = Turbofan("tf")
tf.run_once()

# view
tf.view_system()

# Simulation

The turbofan system has a couple of equations/unknowns to solve. We use `cosapp` non-linear solver for this purpose.

In [ ]:
from cosapp.drivers import NonLinearSolver

## Direct mode
`thrust` is computed from `fuel_W`.

In [ ]:
%%time
tf.add_driver(NonLinearSolver('solver', tol = 1e-6))

# environment conditions
tf.fuel_W = 1.0

# run
tf.run_drivers()

print('fuel_W =', round(tf.fuel_W, 2), 'kg/s')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")

## Control mode
`fuel_W` is computed to match functional request, here the fan rotational speed `N1` value.

In [ ]:
solver = tf.add_driver(NonLinearSolver('solver', tol=1e-6))
solver.extend(tf.design_methods["control_mode"])

# use case
tf.N1 = 5000.

# run
tf.run_drivers()

print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")

# Design mode

## Update technical requirements

Turbofan design characteristics are related to components and physical properties.

In [ ]:
# design solver
solver = tf.add_driver(NonLinearSolver('solver', tol=1e-6))
solver.extend(tf.design_methods['scaling'])

# engine state of the art
tf.core.combustor.Tcomb += 100.0

tf.run_drivers()

print('fan diameter =', round(tf.geom.fan_diameter, 2), 'm')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")

## Update functional requirments

In [ ]:
# engine
tf.thrust += 10e3
tf.run_drivers()

print('fan diameter =', round(tf.geom.fan_diameter, 2), 'm')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")

## Off-design computation after design

Fuel consumption for a given altitude/mach/dtamb and thrust. 

In [ ]:
# off-design mode
solver = tf.add_driver(NonLinearSolver('solver', tol=1e-6))
solver.extend(tf.design_methods["control_mode"])

# use case
tf.init_environment(mach=0.8, alt=10000.0, dtamb = 0.0)
tf.N1 = 5000.0

# run 
tf.run_drivers()

print('pamb =', round(tf.pamb), 'Pa')
print('thrust =', round(tf.thrust/1e3), 'kN')
print('N1 =', round(tf.N1), "rpm")